* ## 인공지능 모델 학습 코드


### 라이브러리 로드
***

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as tl
from tensorflow.keras.models import Sequential
import numpy as np
import PIL.Image as PIL
from glob import glob
from sklearn.model_selection import train_test_split
import pandas as pd
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import cv2

###  GPU 활성화 ( GPU 이용 시 )
***

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


###  이미지 처리 함수
***

In [24]:
def ImageProc(path,size,color):
    img = glob(path)
    img = np.random.permutation(img)
    img = [ PIL.open(img[i]).convert(color).resize(size) for i in range(len(img))]
    img = [np.array(img[i]) for i in range(len(img))]
    img = np.array(img)
    img = img/255.
    return img

###  이미지 로드 및 테스트/학습 데이터셋 분류
***

In [ ]:
Kids = ImageProc('Result/Kids/*.jpg',(96,96),'RGB')
Adults =  ImageProc('Result/Adults/*.jpg',(96,96),'RGB')
Olds =  ImageProc('Result/Olds/*.jpg',(96,96),'RGB')
All_x = np.vstack((Kids,Adults,Olds))
All_y = np.hstack((np.zeros(Kids.shape[0]),np.ones(Adults.shape[0]),np.full(Olds.shape[0],2)))
All_y = pd.get_dummies(All_y)
x,x_test,y,y_test = train_test_split(All_x,All_y,test_size=0.2,random_state=2000)

print('[Dataset] kids: {}, adults: {}, olds: {}'.format(Kids.shape[0]  , Adults.shape[0] , Olds.shape[0]))

###  모델 구성
***

In [26]:
model = Sequential()
model.add(tl.Input((96,96,3)))
model.add(tl.Conv2D(32,kernel_size=3,activation='selu'))
model.add(tl.MaxPool2D(pool_size=2))
model.add(tl.Conv2D(64,kernel_size=3,activation='selu'))
model.add(tl.MaxPool2D(pool_size=2))
model.add(tl.Conv2D(128,kernel_size=3,activation='selu'))
model.add(tl.MaxPool2D(pool_size=2))
model.add(tl.Conv2D(256,kernel_size=3,activation='selu'))
model.add(tl.Flatten())
model.add(tl.Dense(256))
model.add(tl.BatchNormalization())
model.add(tl.Activation(activation='selu'))
model.add(tl.Dropout(0.15))
model.add(tl.Dense(128))
model.add(tl.BatchNormalization())
model.add(tl.Activation(activation='selu'))
model.add(tl.Dense(3,activation='softmax'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 94, 94, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 256)        

In [28]:
model.compile('adam','categorical_crossentropy','accuracy')

###  데이터 증식 및 학습
***

In [29]:
Train_Datagen= ImageDataGenerator(rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
Test_Datagen = ImageDataGenerator()


In [32]:
Train_Gen = Train_Datagen.flow(x,y,batch_size=8)
Test_Gen = Test_Datagen.flow(x_test,y_test,batch_size=8)

In [33]:
early_stopping = EarlyStopping(patience = 20)
model.fit(Train_Gen,epochs=200,validation_data=Test_Gen,callbacks=[early_stopping])

Epoch 1/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1833 - accuracy: 0.9347 - val_loss: 0.2084 - val_accuracy: 0.9227
Epoch 2/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1565 - accuracy: 0.9528 - val_loss: 0.2375 - val_accuracy: 0.9061
Epoch 3/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1641 - accuracy: 0.9361 - val_loss: 0.2053 - val_accuracy: 0.9171
Epoch 4/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1417 - accuracy: 0.9514 - val_loss: 0.2271 - val_accuracy: 0.9171
Epoch 5/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1534 - accuracy: 0.9472 - val_loss: 0.2159 - val_accuracy: 0.9061
Epoch 6/200
90/90 [==============================] - 1s 15ms/step - loss: 0.1437 - accuracy: 0.9583 - val_loss: 0.2197 - val_accuracy: 0.9227
Epoch 7/200
90/90 [==============================] - 1s 16ms/step - loss: 0.1564 - accuracy: 0.9444 - val_loss: 0.2454 - val_accuracy: 0.9171
Epoch 

###  예측 결과 확인 및 저장
***

In [22]:
age_list = ['kid','adult','old']
age_list[model.predict(x_test[36].reshape(1,96,96,3)).argmax()]

'adult'

In [36]:
model.save('Add-on/Mask_Face_Detector_new.h5')